模型初始化测试

In [ ]:
import os
from openai import OpenAI
from langchain.chat_models import ChatOpenAI

client = ChatOpenAI(
        model_name="qwen-plus",
        temperature=0.7,
        streaming=True,
        api_key='sk-1549cf0b1379406fa0d73e3b3d910074', 
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

result = client.invoke("你是谁？")


In [ ]:
print(result)

Chat应用基础层测试

In [1]:
import django_setup

Development settings loaded
INSTALLED_APPS: ['django.contrib.admin', 'django.contrib.auth', 'django.contrib.contenttypes', 'django.contrib.sessions', 'django.contrib.messages', 'django.contrib.staticfiles', 'rest_framework', 'corsheaders', 'storages', 'apps.authentication', 'apps.files', 'apps.projects', 'apps.doc_analysis', 'apps.chat', 'django_filters', 'drf_spectacular', 'rest_framework_simplejwt.token_blacklist']


INFO 2025-02-09 00:51:48,037 storage 使用 COSStorage 初始化存储
INFO 2025-02-09 00:51:48,037 storage 初始化 COSStorage
INFO 2025-02-09 00:51:48,037 storage Storage backend initialization with args: (), kwargs: {}
INFO 2025-02-09 00:51:48,038 storage ✅ 成功初始化 COSStorage
INFO 2025-02-09 00:51:48,038 storage default_storage 的类型: COSStorage


Settings从哪里加载？: config.settings.development
项目根目录对么？: C:\Users\huiwa\Documents\_All_Projects\BidPilot_new\backend
文件存储settings对么？: apps.files.storage.COSStorage
文件default_storage对么？: COSStorage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- apps.projects
- apps.doc_analysis
- apps.chat
- django_filters
- drf_spectacular
- rest_framework_simplejwt.token_blacklist


In [2]:
# 准备测试所需的 user, project, file_record  (其中project与file_record关联)
from django.contrib.auth import get_user_model
User = get_user_model()

# 获取已存在的测试数据

# 获取已存在的用户
test_user = User.objects.get(phone='18501771516')
print(f"用户: {test_user.phone}")


用户: 18501771516


In [3]:
# 1. 测试模型操作
from apps.chat.models import ChatSession, ChatMessage

# 测试会话创建
session = ChatSession.objects.create(created_by=test_user)
print(f"会话: {session.id}")


会话: bd8fd44d-ce2c-49c2-91a1-3a6a92ffd097


In [4]:
from pprint import pprint
pprint(session.__dict__)

{'_state': <django.db.models.base.ModelState object at 0x000002069A591A00>,
 'created_at': datetime.datetime(2025, 2, 8, 16, 51, 50, 753057, tzinfo=datetime.timezone.utc),
 'created_by_id': UUID('1ca30c09-2647-4b8e-b61f-a7ebed81327c'),
 'id': UUID('bd8fd44d-ce2c-49c2-91a1-3a6a92ffd097'),
 'updated_at': datetime.datetime(2025, 2, 8, 16, 51, 50, 753057, tzinfo=datetime.timezone.utc)}


In [5]:
message = ChatMessage.objects.create(
    session=session,
    role='user',
    content='Hello'
)
print(f"消息: {message.id}")

消息: 2c3472dc-d628-4668-a91c-5b57169d502c


In [6]:
pprint(message.__dict__)

{'_state': <django.db.models.base.ModelState object at 0x000002069A840F50>,
 'content': 'Hello',
 'created_at': datetime.datetime(2025, 2, 8, 16, 51, 55, 283703, tzinfo=datetime.timezone.utc),
 'id': UUID('2c3472dc-d628-4668-a91c-5b57169d502c'),
 'role': 'user',
 'sequence': 1,
 'session_id': UUID('bd8fd44d-ce2c-49c2-91a1-3a6a92ffd097')}


In [7]:
# 2. 测试序列化器
from apps.chat.serializers import ChatSessionSerializer, ChatMessageSerializer

# 测试序列化
session_serializer = ChatSessionSerializer(session)
print(session_serializer.data)

# 测试消息序列化
message_serializer = ChatMessageSerializer(message)
print(message_serializer.data)



{'id': 'bd8fd44d-ce2c-49c2-91a1-3a6a92ffd097', 'created_at': '2025-02-08T16:51:50.753057Z', 'updated_at': '2025-02-08T16:51:50.753057Z', 'created_by': {'id': '1ca30c09-2647-4b8e-b61f-a7ebed81327c', 'phone': '18501771516', 'email': None, 'role': 'user'}}
{'id': '2c3472dc-d628-4668-a91c-5b57169d502c', 'session': UUID('bd8fd44d-ce2c-49c2-91a1-3a6a92ffd097'), 'sequence': 1, 'content': 'Hello', 'role': 'user', 'created_at': '2025-02-08T16:51:55.283703Z'}


In [8]:
# 3. 测试 LLM 服务
from apps.chat.services import EnhancedLLMService, ChainBasedChatService

# 测试基本对话
llm_service = EnhancedLLMService()
chat_service = ChainBasedChatService()

# 测试简单对话
response = await chat_service.process_message(
    session_id=str(session.id),
    content="What is Python?"
)
print(response)

HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
{'response': "Hello! Python is a high-level, interpreted programming language known for its clear syntax and readability. It was created by Guido van Rossum and first released in 1991. Python is designed to be easy to read while still being very powerful, which makes it an excellent choice for beginners as well as experienced developers.\n\nPython supports multiple programming paradigms, including procedural, object-oriented, and functional programming. It's widely used in many areas such as web development, data analysis, artificial intelligence, scientific computing, automation, and more. Libraries and frameworks like Django, Flask, Pandas, NumPy, TensorFlow, and PyTorch have further expanded Python's utility across different domains.\n\nOne of the key features of Python is its l